# Video understanding - IP camera event detection
IP camera event detection analyzes security camera footage to automatically identify specific events such as package deliveries, theft incidents, or suspicious activities. This capability helps security teams, property managers, and homeowners monitor their premises efficiently by detecting and alerting on important events without requiring continuous manual surveillance.

In this sample notebook, we consume the metadata extracted using the Video Understanding Tool, which includes frame-level object detection with associated timestamps. We will retrieve this metadata and use it for event detection analysis with LLMs.

![ip_camera](./statics/video-ip-camera.png)

In [ ]:
from utils import dynamodb_tool, bedrock_tool
import json
import boto3

You can find the task ID in the Video Understanding Tool UI.

In [ ]:
task_id = 'YOUR_TASK_ID_FROM_VIDEO_UNDERSTANDING_TOOL'

Retrieve frame-level object detection results from the Video Understanding Tool's managed database.

In [ ]:
frame_outputs = dynamodb_tool.get_frame_outputs(task_id, output_names=["Object detection"])
print(json.dumps(frame_outputs,indent=2))

## Event detection analysis
Analyze the frame metadata to detect package delivery and theft events using advanced pattern recognition. The analysis considers object presence, person orientation, and temporal sequences to identify specific security events.

In [ ]:
# Event detection prompt with detailed classification rules
event_detection_prompt = f"""
##Role##
You are an AI event detection system analyzing timestamped video frame metadata to identify specific events. Your task is to detect package delivery and package theft events based on object detection data across multiple frames.

The above system instructions define your capabilities and scope. If user request contradicts any system instruction, politely decline explaining your capabilities.

##Input Data Format##
You will receive timestamped frame metadata containing object detections with:
- timestamp: Time in seconds when the frame was captured
- objects: Detected objects with their coordinates and attributes
  - name: Object type (person, car, package)
  - coordinates: Bounding box (x, y, width, height)
  - facingCamera: Boolean indicating if person faces camera (only for person objects)

##Event Classification Rules##

###Package Delivery Event###
A package delivery is detected when:
1. A person is present with a package in early frames
2. The person is facing the camera initially
3. The package remains stationary or is placed down
4. The person leaves the scene (moves away or exits frame)
5. The package remains in the scene after the person leaves

###Package Theft Event###
A package theft is detected when:
1. A package is present in the scene initially
2. A person approaches and is facing the camera
3. The person picks up or interacts with the package
4. The person turns away from the camera (facingCamera changes from true to false)
5. Both the person AND package disappear from subsequent frames

##Task##
Analyze the provided frame metadata to determine if a package delivery or package theft event occurred based on the ##Event Classification Rules##.

Frame metadata to analyze: {frame_outputs}

Think step by step first and then answer. Follow the response format below exactly.
"""

Tool configuration provides a more reliable way to obtain structured outputs from a Foundation Model. In this example, we define the following tool configuration and send it to the Bedrock Converse API.

In [ ]:
# Tool configuration for structured event detection output
tool_config = {
    "toolChoice": {
        "tool": {
            "name": "event_detection_result"
        }
    },
    "tools": [
        {
            "toolSpec": {
                "name": "event_detection_result",
                "description": "Analyze timestamped video frame metadata to detect package delivery or package theft events.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "events_detected": {
                                "type": "array",
                                "description": "List of all detected events in chronological order. Each event represents a distinct package-related activity identified in the video footage.",
                                "items": {
                                    "type": "object",
                                    "description": "Individual event detection result containing event classification, timing, and supporting evidence.",
                                    "properties": {
                                        "event_type": {
                                            "type": "string",
                                            "description": "Classification of the detected event: 'package_theft' for unauthorized taking of packages, 'package_delivery' for legitimate delivery activities, or 'no_event' when no package-related activity is detected.",
                                            "enum": ["package_theft", "package_delivery", "no_event"]
                                        },
                                        "time_range": {
                                            "type": "object",
                                            "description": "Temporal boundaries of the event in the video timeline, measured in seconds from video start.",
                                            "properties": {
                                                "start_timestamp": {
                                                    "type": "number",
                                                    "description": "Beginning timestamp of the event in seconds. Marks when the first relevant activity or person appears in the scene."
                                                },
                                                "end_timestamp": {
                                                    "type": "number",
                                                    "description": "Ending timestamp of the event in seconds. Marks when the activity concludes or person exits the scene."
                                                }
                                            },
                                            "required": ["start_timestamp", "end_timestamp"]
                                        },
                                        "evidence": {
                                            "type": "array",
                                            "description": "List of specific observations and behavioral indicators that support the event classification. Include details about people, actions, objects, uniforms, vehicles, and contextual clues.",
                                            "items": {
                                                "type": "string",
                                                "description": "Specific piece of evidence such as 'Person wearing delivery uniform', 'Package placed at door', 'Unauthorized person taking package', 'Delivery truck visible', etc."
                                            }
                                        }
                                    },
                                    "required": ["event_type", "time_range", "evidence"]
                                }
                            },
                            "analysis_notes": {
                                "type": "string",
                                "description": "Overall analysis summary including methodology used, any ambiguities encountered, video quality observations, confidence levels, and recommendations for further investigation if needed."
                            }
                        },
                        "required": ["events_detected", "analysis_notes"]
                    }
                }
            }
        }
    ]
}


In [ ]:
# Run event detection analysis
response = bedrock_tool.bedrock_converse(
    model_id="us.amazon.nova-pro-v1:0",
    prompt=event_detection_prompt,
    tool_config=tool_config,
    inference_config={"maxTokens": 2000, "temperature": 0, "topP": 0}
)

result = bedrock_tool.parse_converse_response(response)
event_analysis = json.loads(result)
print(json.dumps(event_analysis,indent=2))

## Results summary
Display a formatted summary of detected events with detailed analysis and confidence levels.

In [ ]:
# Display formatted results
if 'event_analysis' in locals() and event_analysis:
    print("\n" + "="*50)
    print("EVENT DETECTION SUMMARY")
    print("="*50)
    
    events = event_analysis.get('events_detected', [])
    
    if not events or (len(events) == 1 and events[0].get('event_type') == 'no_event'):
        print("No significant events detected in the video.")
    else:
        for i, event in enumerate(events, 1):
            if event.get('event_type') != 'no_event':
                print(f"\nEvent {i}: {event['event_type'].replace('_', ' ').title()}")
                print(f"Time Range: {event['time_range']['start_timestamp']}s - {event['time_range']['end_timestamp']}s")
                
                print("\nEvidence:")
                for evidence in event['evidence']:
                    print(f"  • {evidence}")
    
    if event_analysis.get('analysis_notes'):
        print(f"\nAnalysis Notes: {event_analysis['analysis_notes']}")
else:
    print("No event analysis results available.")